<a href="https://colab.research.google.com/github/Him2001/Scalable-Fault-Diagnosis-in-Generators/blob/main/Sigmoid_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/FYP

/content/drive/MyDrive/FYP


In [ ]:
!nvidia-smi

Sun Mar 26 05:26:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import glob
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import backend as K
from keras import regularizers
from keras.models import Model,Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from keras.layers import GaussianNoise
import pandas as pd
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

# define the MMD loss function
def mmd_loss(y_true, y_pred, weight=1.0):
    # calculate the MMD loss
    mmd_loss = K.mean(tf.square(tf.reduce_mean(y_pred, axis=0) - tf.reduce_mean(y_true, axis=0)))

    # apply the weight to the MMD loss
    mmd_loss = weight * mmd_loss

    return mmd_loss

# define the binary cross-entropy + MMD loss function
def binary_crossentropy_mmd_loss(y_true, y_pred):
    # define the binary cross-entropy loss
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    bce_loss = K.binary_crossentropy(y_true, y_pred)

    # define the weight for the MMD loss
    mmd_weight = 0.1

    # calculate the MMD loss
    mmd = mmd_loss(y_true, y_pred, weight=mmd_weight)

    # combine the losses
    loss = bce_loss + mmd

    return loss



In [ ]:
df = pd.read_csv(r'3ABCOverlap.csv')
X_train = df.drop(['Detection'], axis = 'columns')
Y_train = df.Detection
del df

df1 = pd.read_csv(r'OTest.csv')
X_test = df1.drop(['Detection'], axis = 'columns')
Y_test = df1.Detection
del df1

In [ ]:
def model_evaluate():

    input_signal = Input(shape=(3000,))
    encoded = Dense(1000, activation='relu', use_bias=True)(input_signal)
    encoded = Dense(500, activation='relu', use_bias=True)(encoded)
    encoded = Dense(250, activation='relu', use_bias=True,activity_regularizer=regularizers.l1(10e-5))(encoded)
    encoded = Dense(125, activation='relu', use_bias=True,activity_regularizer=regularizers.l1(10e-5))(encoded)
    bottleneck = Dense(10,activation='relu', use_bias=True)(encoded)

    # Define decoder layers
    decoded1 = Dense(125,activation='relu',use_bias=True)(bottleneck)
    decoded1 = Dense(250,activation='relu',use_bias=True)(decoded1)
    decoded1 = Dense(500,activation='relu',use_bias=True)(decoded1)
    decoded1 = Dense(1000,activation='relu',use_bias=True)(decoded1)
    decoded = Dense(3000,activation='linear',use_bias=True)(decoded1)
    autoencoder = Model(input_signal, decoded, name='autoencoder')
    # Define autoencoder model
    autoencoder.compile(loss='mse',optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
    autoencoder.fit(X_train,X_train, epochs = 50,shuffle = True,batch_size = 100,verbose  ='auto')

    den = Dense(125, activation='relu',activity_regularizer=regularizers.l1(10e-5))(bottleneck)
    # noise = GaussianNoise(0.1)(den)
    out = Dense(1, activation='sigmoid')(den)
    classification_model = Model(input_signal,out)

    for l1,l2 in zip(classification_model.layers[:6],autoencoder.layers[0:6]):
        l1.set_weights(l2.get_weights())
    for layer in classification_model.layers[0:5]:
        layer.trainable = False

    classification_model.compile(loss=binary_crossentropy_mmd_loss, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

    classification_model.fit(X_train,Y_train, epochs = 50, shuffle = True,batch_size = 100,verbose  ='auto')

    # tf.keras.utils.get_custom_objects()['binary_crossentropy_mmd_loss'] = binary_crossentropy_mmd_loss
    # model = tf.keras.models.load_model(r"C:\Users\Anoop\Desktop\test1\errorhighmodel1.h5", custom_objects={'binary_crossentropy_mmd_loss': binary_crossentropy_mmd_loss})

    ls = classification_model.predict(X_test)
    rs = []
    for i in ls:
        if (i >= 0.5):
            rs.append(1)
        else:
            rs.append(0)
    return(accuracy_score(Y_test, rs))

In [ ]:
a1 = []
n = 0
while (n<1):
    accuracy = model_evaluate()
    if accuracy>0.65:
        a1.append(accuracy*100)
        print(accuracy)
        n +=1
print(a1)